In [1]:
import numpy as np
import lsst.sims.featureScheduler as fs
from lsst.sims.speedObservatory import Speed_observatory
import matplotlib.pylab as plt
import healpy as hp
import timeit

In [2]:
# set up a baseline like survey

nside = fs.set_default_nside(nside=32)

survey_length = 2.2 #365.25*10  # days

# Define what we want the final visit ratio map to look like
years = np.round(survey_length/365.25)
target_map = fs.standard_goals(nside=nside)
filters = ['u', 'g', 'r', 'i', 'z', 'y']
surveys = []

for filtername in filters:
    bfs = []
    bfs.append(fs.M5_diff_basis_function(filtername=filtername, nside=nside))
    bfs.append(fs.Target_map_basis_function(filtername=filtername,
                                            target_map=target_map[filtername],
                                            out_of_bounds_val=hp.UNSEEN, nside=nside))

    bfs.append(fs.North_south_patch_basis_function(zenith_min_alt=50., nside=nside))
    #bfs.append(fs.Zenith_mask_basis_function(maxAlt=78., penalty=-100, nside=nside))
    bfs.append(fs.Slewtime_basis_function(filtername=filtername, nside=nside))
    bfs.append(fs.Strict_filter_basis_function(filtername=filtername))

    weights = np.array([3.0, 0.3, 1., 3., 3.])
    surveys.append(fs.Greedy_survey_fields(bfs, weights, block_size=1, filtername=filtername,
                                           dither=True, nside=nside))

surveys.append(fs.Pairs_survey_scripted([], [], ignore_obs='DD'))

# Set up the DD
dd_surveys = fs.generate_dd_surveys()
surveys.extend(dd_surveys)

scheduler = fs.Core_scheduler(surveys, nside=nside)
observatory = Speed_observatory(nside=nside)
# observatory, scheduler, observations = fs.sim_runner(observatory, scheduler,
#                                                      survey_length=survey_length,
#                                                      filename='feature_baseline_%iyrs.db' % years,
#                                                      delete_past=True)


None


In [3]:
# Check that there's nothing in the queue
len(scheduler.queue) == 0

True

In [5]:
number = 20
scheduler.update_conditions(observatory.return_status())
t = timeit.timeit("scheduler.request_observation()", setup="from __main__ import scheduler", number=number)
print(t/number, ' s')

/home/opsim/repos/sims_ocs/python/lsst/sims/ocs/environment/seeing_model.py:61: RuntimeWarning: invalid value encountered in power
  airmass_correction = numpy.power(airmass, self.AIRMASS_CORRECTION_POWER)


(0.008624446392059327, ' s')


In [6]:
# Let's see what happens if we wipe out the pair queue
scheduler.surveys[6].observing_queue = []
t = timeit.timeit("scheduler.request_observation()", setup="from __main__ import scheduler", number=number)
print(t/number, ' s')

(0.008941447734832764, ' s')


In [7]:
a = scheduler.request_observation()

In [8]:
print(a)

[ ( 0.73936289,  0.0495052,  0.,  30., u'r',  0., 2,  0.,  0.,  0.,  0., 0,  0.,  0.,  0.,  0.,  0.,  0.,  0., u'', 2778, 1)]


In [9]:
observatory.return_status()

{'FWHM_geometric_g': array([        nan,         nan,         nan, ...,  1.03159181,
         1.06571809,  1.08452104]),
 'FWHM_geometric_i': array([        nan,         nan,         nan, ...,  0.90210961,
         0.93195246,  0.94839532]),
 'FWHM_geometric_r': array([        nan,         nan,         nan, ...,  0.95582432,
         0.98744412,  1.00486605]),
 'FWHM_geometric_u': array([        nan,         nan,         nan, ...,  1.11984331,
         1.15688906,  1.17730058]),
 'FWHM_geometric_y': array([        nan,         nan,         nan, ...,  0.83635626,
         0.86402391,  0.87926828]),
 'FWHM_geometric_z': array([        nan,         nan,         nan, ...,  0.86464166,
         0.89324503,  0.90900496]),
 'FWHMeff_g': array([        nan,         nan,         nan, ...,  1.39751492,
         1.44374637,  1.46921905]),
 'FWHMeff_i': array([        nan,         nan,         nan, ...,  1.2659622 ,
         1.30784172,  1.33091657]),
 'FWHMeff_r': array([        nan,         nan,

In [10]:
target_maps = {}
nside = fs.set_default_nside(nside=32)  # Required

target_maps['u'] = fs.generate_goal_map(NES_fraction=0.,
                                        WFD_fraction=0.31, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)
target_maps['g'] = fs.generate_goal_map(NES_fraction=0.2,
                                        WFD_fraction=0.44, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)
target_maps['r'] = fs.generate_goal_map(NES_fraction=0.46,
                                        WFD_fraction=1.0, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)
target_maps['i'] = fs.generate_goal_map(NES_fraction=0.46,
                                        WFD_fraction=1.0, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)
target_maps['z'] = fs.generate_goal_map(NES_fraction=0.4,
                                        WFD_fraction=0.9, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)
target_maps['y'] = fs.generate_goal_map(NES_fraction=0.,
                                        WFD_fraction=0.9, SCP_fraction=0.15,
                                        GP_fraction=0.15, nside=nside,
                                        generate_id_map=True)

filters = ['u', 'g', 'r', 'i', 'z', 'y']
surveys = []

for filtername in filters:
    bfs = []
    bfs.append(fs.M5_diff_basis_function(filtername=filtername, nside=nside))
    # The position of this bfs must be passed to Driver (see furthermore) for logical reasons
    bfs.append(fs.Target_map_basis_function(filtername=filtername,
                                            target_map=target_maps[filtername][0],
                                            id_map=target_maps[filtername][1],
                                            name_list=target_maps[filtername][2],
                                            out_of_bounds_val=hp.UNSEEN, nside=nside))

    bfs.append(fs.North_south_patch_basis_function(zenith_min_alt=50., nside=nside))
    # bfs.append(fs.Zenith_mask_basis_function(maxAlt=78., penalty=-100, nside=nside))
    bfs.append(fs.Slewtime_basis_function(filtername=filtername, nside=nside))
    bfs.append(fs.Strict_filter_basis_function(filtername=filtername))

    weights = np.array([3.0, 0.5, 1., 3., 3.])
#     surveys.append(fs.Greedy_survey_fields(bfs, weights, block_size=1, filtername=filtername, dither=False,
#                                            nside=nside, smoothing_kernel=9,
#                                            tag_fields=True, tag_map=target_maps[filtername][1]))
    surveys.append(fs.Greedy_survey_fields(bfs, weights, block_size=1, filtername=filtername,
                                       dither=True, nside=nside))



scheduler = fs.Core_scheduler(surveys, nside=nside)  # Required
from lsst.sims.speedObservatory import Speed_observatory
import timeit

number = 20
observatory = Speed_observatory(nside=nside)
scheduler.update_conditions(observatory.return_status())
t = timeit.timeit("scheduler.request_observation()", setup="from %s import scheduler" % __name__, number=number)
print(t/number, ' s')

None
(0.008386945724487305, ' s')
